https://qiita.com/SaitoTsutomu/items/a33aba1a95828eb6bd3f

In [2]:
import numpy as np, pandas as pd
from pulp import *
from ortoolpy import addvars, addbinvars

In [3]:
a = pd.read_html('http://qiita.com/shouta-dev/items/1970c2746c3c30f6b39e')[1].T
a,a.columns = a.iloc[1:],a.iloc[0].tolist()
a.必要人数 = a.必要人数.astype(int)
a.iloc[:,2:] = ~a.iloc[:,2:].isnull()
a.insert(0, '曜日', a.index.str[0])
a.reset_index(drop=True, inplace=True)
a = a.iloc[:,list(range(3,a.shape[1]))+[0,1,2]]

In [4]:
a

,従業員0,従業員1,従業員2,従業員3,従業員4,従業員5,従業員6,従業員7,従業員8,従業員9,曜日,時間帯,必要人数
0,True,True,False,True,False,True,False,False,False,False,月,朝,2
1,False,True,False,True,False,True,False,True,False,False,月,昼,3
2,False,True,False,True,True,True,False,False,True,False,月,夜,3
3,True,False,False,True,False,True,False,False,False,False,火,朝,2
4,False,False,False,True,False,True,False,True,False,False,火,昼,3
5,False,False,False,True,True,True,False,False,True,False,火,夜,3
6,True,True,False,True,False,True,False,False,False,False,水,朝,2
7,False,True,False,True,False,True,False,True,False,False,水,昼,3
8,False,True,False,True,True,True,False,False,True,False,水,夜,3
9,True,False,False,True,False,True,False,False,False,True,木,朝,1


In [5]:
Nコマ,N従業員 = a.shape[0],a.shape[1]-3
L従業員 = list(range(N従業員))
L管理者 = [3,5,9] # 管理者は従業員3, 5, 9
C必要人数差 = 10
C希望不可 = 100
C最低コマ数 = 1
C管理者不足 = 100
C1日2コマ = 10

In [6]:
Nコマ

21

In [7]:
m = LpProblem() # 数理モデル
V割当 = np.array(addbinvars(Nコマ, N従業員))
a['V必要人数差'] = addvars(Nコマ)
V最低コマ数 = addvars(N従業員)
a['V管理者不足'] = addvars(Nコマ)
V1日2コマ = addvars(N従業員)
m += (C必要人数差 * lpSum(a.V必要人数差)
    + C希望不可 * lpSum(a.apply(lambda r: lpDot(1-r[L従業員],V割当[r.name]), 1))
    + C最低コマ数 * lpSum(V最低コマ数)
    + C管理者不足 * lpSum(a.V管理者不足)
    + C1日2コマ * lpSum(V1日2コマ)) # 目的関数
for _,r in a.iterrows():
    m += r.V必要人数差 >=  (lpSum(V割当[r.name]) - r.必要人数)
    m += r.V必要人数差 >= -(lpSum(V割当[r.name]) - r.必要人数)
    m += lpSum(V割当[r.name,L管理者]) + r.V管理者不足 >= 1
for j,n in enumerate((a.iloc[:,L従業員].sum(0)+1)//2):
    m += lpSum(V割当[:,j]) + V最低コマ数[j] >= n # 希望の半分以上
for _,v in a.groupby('曜日'):
    for j in range(N従業員):
        m += lpSum(V割当[v.index,j]) + V1日2コマ[j] <= 2 # 各曜日で2コマまで
m

NoName:
MINIMIZE
100*v000003 + 100*v000005 + 100*v000007 + 100*v000008 + 100*v000009 + 100*v000010 + 100*v000011 + 100*v000013 + 100*v000015 + 100*v000017 + 100*v000019 + 100*v000020 + 100*v000021 + 100*v000023 + 100*v000027 + 100*v000028 + 100*v000030 + 100*v000032 + 100*v000033 + 100*v000035 + 100*v000037 + 100*v000038 + 100*v000039 + 100*v000040 + 100*v000041 + 100*v000042 + 100*v000043 + 100*v000045 + 100*v000047 + 100*v000049 + 100*v000050 + 100*v000051 + 100*v000052 + 100*v000053 + 100*v000057 + 100*v000058 + 100*v000060 + 100*v000063 + 100*v000065 + 100*v000067 + 100*v000068 + 100*v000069 + 100*v000070 + 100*v000071 + 100*v000073 + 100*v000075 + 100*v000077 + 100*v000079 + 100*v000080 + 100*v000081 + 100*v000083 + 100*v000087 + 100*v000088 + 100*v000090 + 100*v000092 + 100*v000093 + 100*v000095 + 100*v000097 + 100*v000098 + 100*v000099 + 100*v000101 + 100*v000102 + 100*v000103 + 100*v000105 + 100*v000107 + 100*v000109 + 100*v000111 + 100*v000112 + 100*v000113 + 100*v000117 + 100

In [10]:
type(V割当)

numpy.ndarray

In [11]:
%time m.solve()
R結果 = np.vectorize(value)(V割当).astype(int)
a['結果'] = [''.join(i*j for i,j in zip(r,a.columns)) for r in R結果]
print('目的関数', value(m.objective))
print(a[['曜日','時間帯','結果']])

CPU times: user 17.3 ms, sys: 136 µs, total: 17.4 ms
Wall time: 40.6 ms
目的関数 0.0
   曜日 時間帯                結果
0   月   朝          従業員1従業員5
1   月   昼      従業員3従業員5従業員7
2   月   夜      従業員1従業員3従業員4
3   火   朝          従業員0従業員3
4   火   昼      従業員3従業員5従業員7
5   火   夜      従業員4従業員5従業員8
6   水   朝          従業員0従業員5
7   水   昼      従業員1従業員3従業員5
8   水   夜      従業員3従業員4従業員8
9   木   朝              従業員3
10  木   昼          従業員5従業員7
11  木   夜          従業員8従業員9
12  金   朝          従業員1従業員5
13  金   昼      従業員1従業員7従業員9
14  金   夜      従業員5従業員6従業員8
15  土   朝          従業員0従業員3
16  土   昼  従業員2従業員6従業員7従業員9
17  土   夜  従業員3従業員4従業員6従業員9
18  日   朝          従業員0従業員9
19  日   昼  従業員2従業員3従業員6従業員9
20  日   夜  従業員2従業員3従業員4従業員6
